# 🔬 Biomedical Imaging Technologies: Hands-on Practice

## Table of Contents
1. [Image Loading and Basic Operations](#practice-1-image-loading-and-basic-operations)
2. [Microscopy Image Analysis](#practice-2-microscopy-image-analysis)
3. [Medical Image Processing (DICOM)](#practice-3-medical-image-processing-dicom)
4. [Image Segmentation Techniques](#practice-4-image-segmentation-techniques)
5. [Feature Extraction and Quantification](#practice-5-feature-extraction-and-quantification)
6. [3D Visualization and Reconstruction](#practice-6-3d-visualization-and-reconstruction)

---
**Learning Objectives:**
- Understand digital image fundamentals (pixels, bit depth, file formats)
- Apply preprocessing techniques (filtering, contrast enhancement)
- Implement segmentation methods (thresholding, region growing, watershed)
- Extract quantitative features from biomedical images
- Visualize 3D medical imaging data

## Installing and Importing Essential Libraries

In [ ]:
# Import essential libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import io, filters, exposure, morphology, segmentation, measure, color
from skimage.util import img_as_ubyte, img_as_float
from scipy import ndimage
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
sns.set_style('whitegrid')

print("✅ All libraries loaded successfully!")
print(f"NumPy version: {np.__version__}")
print(f"scikit-image available for image processing")

---
## Practice 1: Image Loading and Basic Operations

### 🎯 Learning Objectives
- Understand pixel and voxel concepts (2D vs 3D)
- Learn about bit depth (8-bit: 256 levels, 16-bit: 65,536 levels)
- Perform basic image operations

### 📖 Key Concepts
- **Pixel**: Picture element (2D)
- **Bit depth**: Number of gray levels
- **Dynamic range**: Difference between brightest and darkest values

In [ ]:
# 1.1 Create a synthetic microscopy image
def create_synthetic_cells(size=256, n_cells=5):
    """Generate synthetic cell images for practice"""
    image = np.zeros((size, size), dtype=np.float32)
    
    np.random.seed(42)
    for _ in range(n_cells):
        # Random cell position
        y, x = np.random.randint(30, size-30, 2)
        radius = np.random.randint(15, 30)
        
        # Create cell
        yy, xx = np.ogrid[:size, :size]
        mask = (yy - y)**2 + (xx - x)**2 <= radius**2
        image[mask] = np.random.uniform(0.6, 1.0)
    
    # Add noise
    noise = np.random.normal(0, 0.05, image.shape)
    image = np.clip(image + noise, 0, 1)
    
    return image

# Create image
image = create_synthetic_cells()

# Display image properties
print("📊 Image Properties")
print("=" * 50)
print(f"Shape: {image.shape}")
print(f"Data type: {image.dtype}")
print(f"Min value: {image.min():.4f}")
print(f"Max value: {image.max():.4f}")
print(f"Mean intensity: {image.mean():.4f}")
print(f"Standard deviation: {image.std():.4f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Original image
im = axes[0].imshow(image, cmap='gray')
axes[0].set_title('Synthetic Cell Image', fontsize=14, fontweight='bold')
axes[0].axis('off')
plt.colorbar(im, ax=axes[0], fraction=0.046)

# Histogram
axes[1].hist(image.ravel(), bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[1].set_title('Intensity Histogram', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Pixel Intensity')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Image created and analyzed successfully!")

---
## Practice 2: Microscopy Image Analysis

### 🎯 Learning Objectives
- Apply noise reduction filters (Gaussian, median, bilateral)
- Perform contrast enhancement
- Understand histogram equalization

### 📖 Key Concepts from Lecture
- **Gaussian filter**: Smoothing with weighted average
- **Median filter**: Removes salt-and-pepper noise
- **Histogram equalization**: Uniform intensity distribution

In [ ]:
# 2.1 Image preprocessing techniques
def apply_preprocessing(image):
    """Apply various preprocessing techniques"""
    
    # 1. Gaussian filtering
    gaussian = filters.gaussian(image, sigma=1.5)
    
    # 2. Median filtering
    median = filters.median(image, morphology.disk(3))
    
    # 3. Histogram equalization
    equalized = exposure.equalize_hist(image)
    
    # 4. Contrast stretching
    p2, p98 = np.percentile(image, (2, 98))
    stretched = exposure.rescale_intensity(image, in_range=(p2, p98))
    
    return gaussian, median, equalized, stretched

# Apply preprocessing
gaussian_img, median_img, eq_img, stretched_img = apply_preprocessing(image)

# Visualize results
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

images = [image, gaussian_img, median_img, stretched_img, eq_img, image]
titles = ['Original', 'Gaussian Filter\n(σ=1.5)', 'Median Filter\n(disk r=3)', 
          'Contrast Stretching\n(2-98 percentile)', 'Histogram\nEqualization', 'Reference']

for ax, img, title in zip(axes.flat, images, titles):
    ax.imshow(img, cmap='gray')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.axis('off')

axes[1, 2].axis('on')
axes[1, 2].hist(image.ravel(), bins=50, alpha=0.5, label='Original', color='blue')
axes[1, 2].hist(eq_img.ravel(), bins=50, alpha=0.5, label='Equalized', color='red')
axes[1, 2].set_title('Histogram Comparison', fontsize=12, fontweight='bold')
axes[1, 2].set_xlabel('Intensity')
axes[1, 2].set_ylabel('Frequency')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Preprocessing techniques applied successfully!")

---
## Practice 3: Medical Image Processing (DICOM)

### 🎯 Learning Objectives
- Understand DICOM format basics
- Apply morphological operations
- Learn erosion, dilation, opening, closing

### 📖 Key Concepts
- **Erosion**: Removes pixels at boundaries
- **Dilation**: Adds pixels at boundaries
- **Opening**: Erosion followed by dilation (removes small objects)
- **Closing**: Dilation followed by erosion (fills small holes)

In [ ]:
# 3.1 Morphological operations
def apply_morphology(image, threshold=0.3):
    """Apply morphological operations for image enhancement"""
    
    # Create binary image
    binary = image > threshold
    
    # Define structuring element
    selem = morphology.disk(3)
    
    # Apply operations
    eroded = morphology.erosion(binary, selem)
    dilated = morphology.dilation(binary, selem)
    opened = morphology.opening(binary, selem)
    closed = morphology.closing(binary, selem)
    
    return binary, eroded, dilated, opened, closed

# Apply morphology
binary, eroded, dilated, opened, closed = apply_morphology(image)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

morph_images = [image, binary, eroded, dilated, opened, closed]
morph_titles = ['Original (Grayscale)', 'Binary\n(threshold=0.3)', 'Erosion\n(removes boundaries)', 
                'Dilation\n(expands boundaries)', 'Opening\n(erosion→dilation)', 'Closing\n(dilation→erosion)']

for ax, img, title in zip(axes.flat, morph_images, morph_titles):
    if title.startswith('Original'):
        ax.imshow(img, cmap='gray')
    else:
        ax.imshow(img, cmap='gray')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()

print("\n📊 Morphological Operations Summary:")
print("=" * 50)
print(f"Binary image: {binary.sum()} white pixels")
print(f"Eroded: {eroded.sum()} white pixels (smaller objects)")
print(f"Dilated: {dilated.sum()} white pixels (larger objects)")
print(f"Opened: {opened.sum()} white pixels (noise removed)")
print(f"Closed: {closed.sum()} white pixels (holes filled)")
print("\n✅ Morphological operations completed!")

---
## Practice 4: Image Segmentation Techniques

### 🎯 Learning Objectives
- Implement multiple segmentation methods
- Compare thresholding, watershed, and region growing
- Understand when to use each method

### 📖 Key Concepts from Lecture
- **Otsu's method**: Automatic threshold selection
- **Watershed**: Treats image as topographic surface
- **Region growing**: Groups similar pixels from seed points

In [ ]:
# 4.1 Multiple segmentation techniques
def compare_segmentation_methods(image):
    """Compare different segmentation approaches"""
    
    print("🔍 Applying Segmentation Methods...")
    print("=" * 50)
    
    # 1. Otsu's thresholding
    threshold_value = filters.threshold_otsu(image)
    otsu_seg = image > threshold_value
    print(f"1. Otsu threshold value: {threshold_value:.4f}")
    
    # 2. Adaptive thresholding
    adaptive_seg = image > filters.threshold_local(image, block_size=35)
    print("2. Adaptive thresholding applied (block_size=35)")
    
    # 3. Watershed segmentation
    # Compute elevation map
    elevation_map = filters.sobel(image)
    
    # Find markers
    markers = np.zeros_like(image, dtype=int)
    markers[image < 0.2] = 1  # Background
    markers[image > 0.6] = 2  # Foreground
    
    watershed_seg = segmentation.watershed(elevation_map, markers)
    print("3. Watershed segmentation completed")
    
    # 4. Connected component labeling
    labeled_image = measure.label(otsu_seg)
    n_objects = labeled_image.max()
    print(f"4. Found {n_objects} connected objects")
    
    return otsu_seg, adaptive_seg, watershed_seg, labeled_image, n_objects

# Apply segmentation
otsu_result, adaptive_result, watershed_result, labeled, n_cells = compare_segmentation_methods(image)

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Original
axes[0, 0].imshow(image, cmap='gray')
axes[0, 0].set_title('Original Image', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# Otsu
axes[0, 1].imshow(otsu_result, cmap='gray')
axes[0, 1].set_title('Otsu Thresholding\n(Global)', fontsize=12, fontweight='bold')
axes[0, 1].axis('off')

# Adaptive
axes[0, 2].imshow(adaptive_result, cmap='gray')
axes[0, 2].set_title('Adaptive Thresholding\n(Local)', fontsize=12, fontweight='bold')
axes[0, 2].axis('off')

# Watershed
axes[1, 0].imshow(watershed_result, cmap='nipy_spectral')
axes[1, 0].set_title('Watershed Segmentation', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')

# Labeled
axes[1, 1].imshow(color.label2rgb(labeled, bg_label=0))
axes[1, 1].set_title(f'Connected Components\n({n_cells} objects)', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')

# Overlay
axes[1, 2].imshow(image, cmap='gray')
axes[1, 2].contour(otsu_result, colors='red', linewidths=2)
axes[1, 2].set_title('Original + Boundaries', fontsize=12, fontweight='bold')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ Segmentation comparison completed!")

---
## Practice 5: Feature Extraction and Quantification

### 🎯 Learning Objectives
- Extract shape descriptors (area, perimeter, circularity)
- Calculate intensity statistics
- Generate quantitative reports

### 📖 Key Concepts
- **Area**: Number of pixels in object
- **Perimeter**: Boundary length
- **Circularity**: $4\pi \times \frac{Area}{Perimeter^2}$
- **Intensity metrics**: Mean, std, min/max

In [ ]:
# 5.1 Extract quantitative features
def extract_features(image, labeled_image):
    """Extract shape and intensity features from labeled objects"""
    
    print("📊 Extracting Features from Detected Objects...")
    print("=" * 60)
    
    # Get region properties
    regions = measure.regionprops(labeled_image, intensity_image=image)
    
    features = []
    for i, region in enumerate(regions, 1):
        feature = {
            'Object': i,
            'Area (pixels)': region.area,
            'Perimeter (pixels)': region.perimeter,
            'Circularity': 4 * np.pi * region.area / (region.perimeter ** 2) if region.perimeter > 0 else 0,
            'Mean Intensity': region.mean_intensity,
            'Max Intensity': region.max_intensity,
            'Centroid Y': region.centroid[0],
            'Centroid X': region.centroid[1]
        }
        features.append(feature)
    
    # Create DataFrame
    import pandas as pd
    df = pd.DataFrame(features)
    
    print(f"\nTotal objects detected: {len(features)}")
    print("\nFeature Summary Statistics:")
    print(df.describe())
    
    return df, regions

# Extract features
import pandas as pd
feature_df, detected_regions = extract_features(image, labeled)

# Visualize features
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Image with labels
axes[0].imshow(image, cmap='gray')
for region in detected_regions:
    y, x = region.centroid
    axes[0].plot(x, y, 'r+', markersize=15, markeredgewidth=2)
    axes[0].text(x, y-20, f'{region.label}', color='red', fontsize=10, 
                fontweight='bold', ha='center')
axes[0].set_title('Detected Objects with Labels', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Area distribution
axes[1].bar(feature_df['Object'], feature_df['Area (pixels)'], color='steelblue', edgecolor='black')
axes[1].set_xlabel('Object ID', fontsize=11)
axes[1].set_ylabel('Area (pixels)', fontsize=11)
axes[1].set_title('Object Area Distribution', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Circularity vs Area
scatter = axes[2].scatter(feature_df['Area (pixels)'], feature_df['Circularity'], 
                         c=feature_df['Mean Intensity'], cmap='viridis', s=100, 
                         edgecolor='black', linewidth=1.5, alpha=0.7)
axes[2].set_xlabel('Area (pixels)', fontsize=11)
axes[2].set_ylabel('Circularity', fontsize=11)
axes[2].set_title('Shape Analysis (color=intensity)', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[2], label='Mean Intensity')

plt.tight_layout()
plt.show()

print("\n✅ Feature extraction completed!")
print("\n📋 Sample Feature Table:")
print(feature_df.head().to_string(index=False))

---
## Practice 6: 3D Visualization and Reconstruction

### 🎯 Learning Objectives
- Create 3D image stacks
- Apply maximum intensity projection (MIP)
- Visualize multi-planar reformation (MPR)

### 📖 Key Concepts
- **Z-stack**: Series of 2D images at different depths
- **MIP**: Shows brightest voxels along viewing direction
- **MPR**: Creates arbitrary slice planes from 3D data

In [ ]:
# 6.1 Create and visualize 3D image stack
def create_3d_stack(n_slices=10):
    """Generate synthetic 3D image stack"""
    
    print("🔨 Creating 3D Image Stack...")
    print("=" * 50)
    
    stack = []
    for z in range(n_slices):
        # Create slice with varying intensity
        slice_img = create_synthetic_cells(size=128, n_cells=3 + z % 3)
        # Add depth-dependent intensity variation
        depth_factor = 1.0 - abs(z - n_slices/2) / (n_slices/2) * 0.3
        slice_img = slice_img * depth_factor
        stack.append(slice_img)
    
    stack_3d = np.array(stack)
    print(f"Stack shape: {stack_3d.shape} (Z, Y, X)")
    print(f"Total voxels: {stack_3d.size:,}")
    
    return stack_3d

# Create stack
image_stack = create_3d_stack(n_slices=10)

# Maximum Intensity Projection
mip_z = np.max(image_stack, axis=0)  # Along Z
mip_y = np.max(image_stack, axis=1)  # Along Y
mip_x = np.max(image_stack, axis=2)  # Along X

# Visualize
fig = plt.figure(figsize=(16, 10))

# Individual slices
for i in range(6):
    ax = plt.subplot(3, 4, i+1)
    slice_idx = i * 2
    ax.imshow(image_stack[slice_idx], cmap='gray')
    ax.set_title(f'Slice Z={slice_idx}', fontsize=11, fontweight='bold')
    ax.axis('off')

# MIP projections
ax = plt.subplot(3, 4, 7)
ax.imshow(mip_z, cmap='hot')
ax.set_title('MIP (Z-axis)\nTop view', fontsize=11, fontweight='bold')
ax.axis('off')

ax = plt.subplot(3, 4, 8)
ax.imshow(mip_y, cmap='hot')
ax.set_title('MIP (Y-axis)\nFront view', fontsize=11, fontweight='bold')
ax.axis('off')

ax = plt.subplot(3, 4, 11)
ax.imshow(mip_x, cmap='hot')
ax.set_title('MIP (X-axis)\nSide view', fontsize=11, fontweight='bold')
ax.axis('off')

# 3D visualization representation
ax = plt.subplot(3, 4, 12)
middle_slice = image_stack[image_stack.shape[0]//2]
ax.imshow(middle_slice, cmap='gray')
ax.contour(middle_slice, levels=3, colors='cyan', linewidths=2)
ax.set_title('Middle Slice\n+ Contours', fontsize=11, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.show()

print("\n📊 3D Stack Analysis:")
print("=" * 50)
print(f"Dimensions: {image_stack.shape[0]} slices × {image_stack.shape[1]} × {image_stack.shape[2]} pixels")
print(f"Voxel intensities: min={image_stack.min():.4f}, max={image_stack.max():.4f}")
print(f"Mean intensity per slice: {[f'{s.mean():.3f}' for s in image_stack[:5]]}...")
print("\n✅ 3D visualization completed!")

---
## 🎯 Practice Complete!

### Summary of What We Learned:

1. **Digital Image Basics**: Pixels, bit depth, intensity distributions
2. **Preprocessing**: Filtering, contrast enhancement, morphological operations
3. **Segmentation**: Otsu, adaptive thresholding, watershed algorithm
4. **Feature Extraction**: Shape descriptors, intensity statistics, quantification
5. **3D Imaging**: Stack creation, MIP, multi-planar views

### Key Insights:
- Image preprocessing is crucial for accurate analysis
- Different segmentation methods work for different scenarios
- Quantitative features enable objective image analysis
- 3D visualization helps understand complex structures

### Clinical Applications:
- **Microscopy**: Cell counting, morphology analysis
- **Radiology**: Tumor segmentation, volume measurement
- **Pathology**: Tissue classification, cancer detection
- **Surgery Planning**: 3D reconstruction, anatomical visualization

### Next Steps:
- Explore real DICOM medical images
- Implement machine learning-based segmentation (U-Net)
- Practice with ImageJ and 3D Slicer software
- Learn about image registration for multi-modal fusion

---
## 📚 Additional Resources

### Python Libraries:
- **scikit-image**: Image processing in Python - https://scikit-image.org/
- **SimpleITK**: Medical image analysis - https://simpleitk.org/
- **pydicom**: DICOM file handling - https://pydicom.github.io/

### Software Tools:
- **ImageJ/Fiji**: Free image processing - https://fiji.sc/
- **3D Slicer**: Medical image visualization - https://www.slicer.org/
- **QuPath**: Digital pathology analysis - https://qupath.github.io/

### Datasets for Practice:
- **Cell Image Library**: http://www.cellimagelibrary.org/
- **The Cancer Imaging Archive (TCIA)**: https://www.cancerimagingarchive.net/
- **BioImage Archive**: https://www.ebi.ac.uk/bioimage-archive/

### Further Reading:
- Gonzalez & Woods - "Digital Image Processing"
- Sonka et al. - "Image Processing, Analysis, and Machine Vision"
- Bankman - "Handbook of Medical Image Processing"